In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules


C:\Users\subasini\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv("./order_item_history.csv",encoding="latin")
data.head(5)

,ï»¿id,order_id,order_item_id,item_id,item_name,staff_id,initial_quantity,updated_quantity,current_quantity,price,tax_fees,subtotal,action,cancel_reason,created_time
0,0001720a-e623-4fe2-a6d0-0b855ef9d8b6,83529c2a-a139-4b9b-acfb-ab641036b352,16c4d5a9-9045-450a-8753-b810e75e4965,f4005a91-fc71-4fa6-b659-2bcc3ddea42f,Omapodi,NaN,4,0,4,4.49,NaN,0.0,ADD,NaN,27/11/2023 07:23
1,00024738-3ac6-482b-b9ad-5e9e5be9a68c,4ee160f5-b191-43e7-a4ce-b3e2acde92ac,d08fc2c5-37ca-4dd3-b64c-3c42805ea41a,f2a2c27f-7f30-489d-9a87-061fd583f740,Vegetable Laba,154eca85-b2d7-4b9d-892b-97c960d44786,1,0,1,15.00,NaN,0.0,ADD,NaN,30/01/2024 04:59
2,0003a175-7a37-44f6-b447-ce8f168374aa,a2004bfa-1161-4129-8cd2-ab48e1652032,b450ea8c-d741-4b2a-a0eb-42c361654ae1,ec5ea9ae-b435-434e-8767-d50531e7298c,Veg Kothu Parotta,154eca85-b2d7-4b9d-892b-97c960d44786,3,0,3,15.99,NaN,0.0,ADD,NaN,14/11/2023 10:28
3,00061855-0ae2-4c33-ba25-d1c1c0f37a82,f9d2a408-24c8-4d9c-a647-d4937fc8da75,b055d736-b301-4cc2-b542-f2d58513ad3d,ec5ea9ae-b435-434e-8767-d50531e7298c,Veg Kothu Parotta,154eca85-b2d7-4b9d-892b-97c960d44786,2,0,2,15.99,NaN,0.0,ADD,NaN,23/10/2023 17:36
4,0008eb77-73b9-4265-bf4d-7ba88a2be961,bd10ccb1-7050-496f-baa9-d47b6a003e77,875d3793-2d36-40cb-8658-046dc5ed2be1,3f810528-dbe0-4ca4-a47a-33ddd2713a3d,Medhu Vadai,11c89ddb-fb4f-4bca-82af-28cc3fa63daa,5,0,5,8.49,NaN,0.0,ADD,NaN,02/11/2023 09:31


In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit(data['item_name'].apply(lambda x: [x]).tolist()).transform(data['item_name'].apply(lambda x: [x]).tolist())
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.0001, use_colnames=True)

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 1]  # consider only itemsets of length >= 2
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Display frequently bought items
print("Frequently bought itemsets:")
for idx, itemset in frequent_itemsets.iterrows():
    print(f"Itemset: {itemset['itemsets']}, Support: {itemset['support']}")

Frequently bought itemsets:
Itemset: frozenset({' Vegetable Laba'}), Support: 0.27075
Itemset: frozenset({' Veg Kothu Parotta'}), Support: 0.15675
Itemset: frozenset({' Veg Fried Rice'}), Support: 0.055
Itemset: frozenset({' Dosai'}), Support: 0.0495
Itemset: frozenset({' Schezwan Fried Rice'}), Support: 0.042
Itemset: frozenset({'Veg Clear Soup'}), Support: 0.027
Itemset: frozenset({'Rasam'}), Support: 0.02225
Itemset: frozenset({' Vegetable Omelette'}), Support: 0.02175
Itemset: frozenset({'Coke'}), Support: 0.0205
Itemset: frozenset({'Cream Of Tomato Soup'}), Support: 0.016
Itemset: frozenset({'Parotta Kurma'}), Support: 0.01225
Itemset: frozenset({'Bonda'}), Support: 0.00775
Itemset: frozenset({'Chapati Kurma'}), Support: 0.0065
Itemset: frozenset({'Idli'}), Support: 0.0065
Itemset: frozenset({'Cream Of Vegetable Soup'}), Support: 0.0065
Itemset: frozenset({'Poori Masala'}), Support: 0.0055
Itemset: frozenset({'Medhu Vadai'}), Support: 0.0055
Itemset: frozenset({'Open Item'}), Supp

In [4]:
order_items_mapping = data.groupby('order_id')['item_name'].agg(list)

for order_id, item_names in order_items_mapping.items():
    print(f"{order_id}-{item_names}")

0017261f-19d5-40c4-8ab5-49705e64debe-['Chapati Kurma']
003fff69-f2a0-4628-83f4-42625fbb3cd5-['Onion Rava Dosai']
005678b0-9390-4c35-be00-8dbf9ab53668-[' Schezwan Fried Rice']
0064201d-1b2f-42c7-be75-a6f2e4db2f41-['Idli', 'Idli']
0067d963-d012-40ea-b4a2-5fbb6eddee12-[' Vegetable Laba', ' Vegetable Laba']
007ad02a-f330-492c-888d-3e06473e5f24-[' Vegetable Laba']
008b5813-4d90-4b49-a26d-0630294a6f9a-[' Veg Kothu Parotta', 'Rasam']
008fe74f-84a9-4e72-ad3c-669130080f68-[' Dosai']
009d3d3d-e231-419a-a159-1d91b99d04fb-['Veg Clear Soup', ' Vegetable Omelette']
00ab9bdc-561d-4c53-8844-d563693d6fc6-['Cream Of Tomato Soup', ' Vegetable Laba']
00b14127-29e7-4167-a4f8-e7d0afaf164c-[' Veg Kothu Parotta']
00e5d484-5e54-41da-84ba-841e1913ca09-['Keerai Vadai']
00f24cdc-e4f9-4b95-9b01-28ef37b58e4d-[' Schezwan Fried Rice']
00f95bce-dde9-436b-bb98-92f2e010c9ef-[' Vegetable Laba']
011205a8-4fa2-442e-a57d-f67848709e00-['Farmers Vegetable Salad']
011ecc27-f68c-41da-b0b3-125ba2935159-['Chikku Milkshake', 'Masa

In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

basket = (data.groupby(['order_id', 'item_name'])['item_name']
          .apply(lambda x: ','.join(x))
          .unstack().reset_index().fillna(0)
          .set_index('order_id'))

def encode_units(x):
    if x == 0:
        return 0
    if x != 0:
        return 1

basket_sets = basket.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

print(rules)
rules.to_csv('./association_rules.csv', index=False)

C:\Users\subasini\AppData\Local\Temp\ipykernel_23648\791510541.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)
C:\Users\subasini\AppData\Roaming\Python\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                       antecedents  \
0                    (Aloo Kulcha)   
1                    (Aloo Kulcha)   
2                    (Aloo Kulcha)   
3                    (Aloo Kulcha)   
4                    (Aloo Kulcha)   
...                            ...   
52184      (Parotta, Masala Dosai)   
52185     (Butter Naan, Dal Tadka)   
52186  (Masala Dosai, Butter Naan)   
52187    (Masala Dosai, Dal Tadka)   
52188                (Aloo Kulcha)   

                                             consequents  antecedent support  \
0                                          (Butter Naan)            0.000671   
1                                       (Chilli Parotta)            0.000671   
2                                            (Dal Tadka)            0.000671   
3                                          (Garlic Naan)            0.000671   
4                                 (Madras Filter Coffee)            0.000671   
...                                                  ...       

In [6]:
aso = pd.read_csv("association_rules.csv",encoding='latin')
aso.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'zhangs_metric'],
      dtype='object')

In [11]:
# Get antecedent items from user input
antecedent_item_input = input("Enter the antecedent item(s) separated by commas: ").strip()
antecedent_items = [item.strip() for item in antecedent_item_input.split(",")]

# Set the threshold value for consequent support
threshold = 0.0002  # Adjust the threshold value as needed

# Filter consequents based on antecedent items and threshold
filtered_rules = rules[rules['antecedents'].apply(lambda x: all(item in str(x) for item in antecedent_items)) & (rules['consequent support'] > threshold)]

# Get consequent items
consequent_items = filtered_rules['consequents'].apply(lambda x: str(x).replace("frozenset({'", "").replace("'})", ""))

# Print consequent items associated with the antecedent items
print("Consequent items associated with '{}':".format(', '.join(antecedent_items)))
print(consequent_items)


Consequent items associated with 'Butter Naan, Dal Tadka':
36                                             Aloo Kulcha
178                                         Chilli Parotta
201                                            Garlic Naan
204                                   Madras Filter Coffee
207                                           Masala Dosai
                               ...                        
52131    Madras Filter Coffee', 'Aloo Kulcha', 'Chilli ...
52137    Madras Filter Coffee', 'Aloo Kulcha', 'Chilli ...
52140    Madras Filter Coffee', 'Aloo Kulcha', 'Chilli ...
52143    Madras Filter Coffee', 'Aloo Kulcha', 'Chilli ...
52185    Madras Filter Coffee', 'Aloo Kulcha', 'Chilli ...
Name: consequents, Length: 6305, dtype: object


In [12]:
# Example output of consequent items associated with the antecedent

# Function to parse consequent items and extract unique items
def extract_unique_items(consequent_items_output):
    unique_items = set()  # Using a set to automatically handle uniqueness
    for item in consequent_items_output:
        # Split the string based on single quote and comma
        items = item.split("', '")
        # Remove leading and trailing single quotes and add to unique_items set
        for i in items:
            unique_items.add(i.strip("'"))
    return list(unique_items)

# Extract unique consequent items
unique_consequent_items = extract_unique_items(consequent_items)

print("Unique consequent items:")
print(unique_consequent_items)


Unique consequent items:
['Madras Filter Coffee', 'Aloo Kulcha', 'Garlic Naan', 'Chilli Parotta', 'Onion Kulcha', 'Roti', 'Parotta', 'Masala Dosai']


In [13]:
from itertools import combinations
# Generate combinations of consequent items
antecedent_combinations = []
for r in range(1, len(unique_consequent_items) + 1):
    antecedent_combinations.extend(combinations(unique_consequent_items, r))

# Print the generated combinations
for combination in antecedent_combinations:
    print(combination)

('Madras Filter Coffee',)
('Aloo Kulcha',)
('Garlic Naan',)
('Chilli Parotta',)
('Onion Kulcha',)
('Roti',)
('Parotta',)
('Masala Dosai',)
('Madras Filter Coffee', 'Aloo Kulcha')
('Madras Filter Coffee', 'Garlic Naan')
('Madras Filter Coffee', 'Chilli Parotta')
('Madras Filter Coffee', 'Onion Kulcha')
('Madras Filter Coffee', 'Roti')
('Madras Filter Coffee', 'Parotta')
('Madras Filter Coffee', 'Masala Dosai')
('Aloo Kulcha', 'Garlic Naan')
('Aloo Kulcha', 'Chilli Parotta')
('Aloo Kulcha', 'Onion Kulcha')
('Aloo Kulcha', 'Roti')
('Aloo Kulcha', 'Parotta')
('Aloo Kulcha', 'Masala Dosai')
('Garlic Naan', 'Chilli Parotta')
('Garlic Naan', 'Onion Kulcha')
('Garlic Naan', 'Roti')
('Garlic Naan', 'Parotta')
('Garlic Naan', 'Masala Dosai')
('Chilli Parotta', 'Onion Kulcha')
('Chilli Parotta', 'Roti')
('Chilli Parotta', 'Parotta')
('Chilli Parotta', 'Masala Dosai')
('Onion Kulcha', 'Roti')
('Onion Kulcha', 'Parotta')
('Onion Kulcha', 'Masala Dosai')
('Roti', 'Parotta')
('Roti', 'Masala Dosai')


In [14]:
from itertools import combinations


# Generate combinations of consequent items
antecedent_combinations = []
for r in range(1, len(unique_consequent_items) + 1):
    antecedent_combinations.extend(combinations(unique_consequent_items, r))

threshold = 0.002  # Adjust the threshold value as needed

# Iterate over the combinations and determine consequents for each combination
for combination in antecedent_combinations:
    # Filter consequents based on the combination
    filtered_rules = rules[rules['antecedents'].apply(lambda x: all(item in str(x) for item in combination))& (rules['consequent support'] > threshold)]

    # Get consequent items
    consequent_items = filtered_rules['consequents'].apply(lambda x: str(x).replace("frozenset({'", "").replace("'})", ""))

    # Print consequent items associated with the antecedent item
    print("Consequent items associated with '{}':".format(', '.join(combination)))
    print(consequent_items)


Consequent items associated with 'Madras Filter Coffee':
26       Strawberry Milkshake
114               Garlic Naan
218               Garlic Naan
280               Garlic Naan
324               Garlic Naan
                 ...         
49180             Garlic Naan
49681             Garlic Naan
50182             Garlic Naan
50684             Garlic Naan
51184             Garlic Naan
Name: consequents, Length: 256, dtype: object
Consequent items associated with 'Aloo Kulcha':
3                 Garlic Naan
4        Madras Filter Coffee
39                Garlic Naan
44       Madras Filter Coffee
67                Garlic Naan
                 ...         
49683    Madras Filter Coffee
50182             Garlic Naan
50184    Madras Filter Coffee
51184             Garlic Naan
51186    Madras Filter Coffee
Name: consequents, Length: 512, dtype: object
Consequent items associated with 'Garlic Naan':
21             Vegetable Laba
23        Paneer Masala Dosai
116      Madras Filter Coffee
219  